In [56]:
import pyspark
from pyspark.sql import functions as F

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/19 09:20:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
import pandas as pd
import numpy as np

In [5]:
lan_dataframe = pd.DataFrame(dict(language=('python','SQL','Tableau')))
lan_dataframe

,language
0,python
1,SQL
2,Tableau


In [6]:
df = spark.createDataFrame(lan_dataframe)

In [7]:
df

DataFrame[language: string]

In [77]:
df.show()

+--------+
|language|
+--------+
|  python|
|     SQL|
| Tableau|
+--------+



In [78]:
df.printSchema()

root
 |-- language: string (nullable = true)



In [79]:
df.count()

3

In [29]:
from pydataset import data
mpg = data('mpg')

In [91]:
mpgdf = spark.createDataFrame(data('mpg'))

In [92]:
mpgdf.describe().show()

+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|summary|manufacturer|            model|             displ|             year|              cyl|     trans|drv|               cty|              hwy|  fl|  class|
+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|  count|         234|              234|               234|              234|              234|       234|234|               234|              234| 234|    234|
|   mean|        null|             null| 3.471794871794872|           2003.5|5.888888888888889|      null|4.0|16.858974358974358|23.44017094017094|null|   null|
| stddev|        null|             null|1.2919590310839348|4.509646313320436|1.611534484684289|      null|0.0| 4.255945678889394|5.954643441166448|null|   null|
|    min|        audi|      4runne

In [93]:
mpgdf.select(
    'statement',
    F.concat(
        F.lit('The '),
        mpg.year,
        F.lit(' '),
        mpg.manufacturer,
        F.lit(' '),
        mpg.model,
        F.lit(' has a'),
        mpg.cyl,
        F.lit(' cylinder engine')
        
    )
).select('statement').shoe(5,turncate=False)

TypeError: Invalid argument, not a string or column: 1      1999
2      1999
3      2008
4      2008
5      1999
       ... 
230    2008
231    2008
232    1999
233    1999
234    2008
Name: year, Length: 234, dtype: int64 of type <class 'pandas.core.series.Series'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [94]:
mpgdf.withColumn(
    'statement',
    F.concat(
        F.lit('The '),
        mpg.year,
        F.lit(' '),
        mpg.manufacturer,
        F.lit(' '),
        mpg.model,
        F.lit(' has a'),
        mpg.cyl,
        F.lit(' cylinder engine')
        
    )
).select('statement').shoe(5,turncate=False)

TypeError: Invalid argument, not a string or column: 1      1999
2      1999
3      2008
4      2008
5      1999
       ... 
230    2008
231    2008
232    1999
233    1999
234    2008
Name: year, Length: 234, dtype: int64 of type <class 'pandas.core.series.Series'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [95]:
mpgdf.select(
    F.when(
        mpg.trans.startswith('auto'),'auto'
    ).otherwise('manual')
)

AttributeError: 'Series' object has no attribute 'startswith'

In [96]:
from pydataset import data
tips = data('tips')

In [97]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [98]:
tipdf = spark.createDataFrame(data('tips'))

In [99]:
tipdf.describe().show()

+-------+------------------+------------------+------+------+----+------+------------------+
|summary|        total_bill|               tip|   sex|smoker| day|  time|              size|
+-------+------------------+------------------+------+------+----+------+------------------+
|  count|               244|               244|   244|   244| 244|   244|               244|
|   mean|19.785942622950813|2.9982786885245907|  null|  null|null|  null| 2.569672131147541|
| stddev| 8.902411954856856| 1.383638189001182|  null|  null|null|  null|0.9510998047322344|
|    min|              3.07|               1.0|Female|    No| Fri|Dinner|                 1|
|    max|             50.81|              10.0|  Male|   Yes|Thur| Lunch|                 6|
+-------+------------------+------------------+------+------+----+------+------------------+



In [101]:
(tipdf.filter(tipdf.smoker=='Yes').count()/tipdf.count())*100

38.114754098360656

In [102]:
(tipdf.filter(tipdf.smoker=='No').count()/tipdf.count())*100

61.885245901639344

In [103]:
tipdf.groupby(tipdf.smoker).agg(
F.round((F.count('time')/tipdf.count()),2).alias('perc')).show()

+------+----+
|smoker|perc|
+------+----+
|    No|0.62|
|   Yes|0.38|
+------+----+



In [105]:
tipdf.withColumn(
    'tip_perc',
    F.round((tipdf.tip/tips.total_bill)*100,2)
).select('tip','total_bill','tip_perc')

SparkRuntimeException: [UNSUPPORTED_FEATURE.LITERAL_TYPE] The feature is not supported: Literal for '{1=16.99, 2=10.34, 3=21.01, 4=23.68, 5=24.59, 6=25.29, 7=8.77, 8=26.88, 9=15.04, 10=14.78, 11=10.27, 12=35.26, 13=15.42, 14=18.43, 15=14.83, 16=21.58, 17=10.33, 18=16.29, 19=16.97, 20=20.65, 21=17.92, 22=20.29, 23=15.77, 24=39.42, 25=19.82, 26=17.81, 27=13.37, 28=12.69, 29=21.7, 30=19.65, 31=9.55, 32=18.35, 33=15.06, 34=20.69, 35=17.78, 36=24.06, 37=16.31, 38=16.93, 39=18.69, 40=31.27, 41=16.04, 42=17.46, 43=13.94, 44=9.68, 45=30.4, 46=18.29, 47=22.23, 48=32.4, 49=28.55, 50=18.04, 51=12.54, 52=10.29, 53=34.81, 54=9.94, 55=25.56, 56=19.49, 57=38.01, 58=26.41, 59=11.24, 60=48.27, 61=20.29, 62=13.81, 63=11.02, 64=18.29, 65=17.59, 66=20.08, 67=16.45, 68=3.07, 69=20.23, 70=15.01, 71=12.02, 72=17.07, 73=26.86, 74=25.28, 75=14.73, 76=10.51, 77=17.92, 78=27.2, 79=22.76, 80=17.29, 81=19.44, 82=16.66, 83=10.07, 84=32.68, 85=15.98, 86=34.83, 87=13.03, 88=18.28, 89=24.71, 90=21.16, 91=28.97, 92=22.

In [107]:
tipdf.withColumn(
    'tip_perc',
    F.round((tipdf.tip/tips.total_bill)*100,2)
).groupby('sex','smoker').agg(
    F.round(F.mean('tip_perc'),2)).alias('avg_perc').show()

SparkRuntimeException: [UNSUPPORTED_FEATURE.LITERAL_TYPE] The feature is not supported: Literal for '{1=16.99, 2=10.34, 3=21.01, 4=23.68, 5=24.59, 6=25.29, 7=8.77, 8=26.88, 9=15.04, 10=14.78, 11=10.27, 12=35.26, 13=15.42, 14=18.43, 15=14.83, 16=21.58, 17=10.33, 18=16.29, 19=16.97, 20=20.65, 21=17.92, 22=20.29, 23=15.77, 24=39.42, 25=19.82, 26=17.81, 27=13.37, 28=12.69, 29=21.7, 30=19.65, 31=9.55, 32=18.35, 33=15.06, 34=20.69, 35=17.78, 36=24.06, 37=16.31, 38=16.93, 39=18.69, 40=31.27, 41=16.04, 42=17.46, 43=13.94, 44=9.68, 45=30.4, 46=18.29, 47=22.23, 48=32.4, 49=28.55, 50=18.04, 51=12.54, 52=10.29, 53=34.81, 54=9.94, 55=25.56, 56=19.49, 57=38.01, 58=26.41, 59=11.24, 60=48.27, 61=20.29, 62=13.81, 63=11.02, 64=18.29, 65=17.59, 66=20.08, 67=16.45, 68=3.07, 69=20.23, 70=15.01, 71=12.02, 72=17.07, 73=26.86, 74=25.28, 75=14.73, 76=10.51, 77=17.92, 78=27.2, 79=22.76, 80=17.29, 81=19.44, 82=16.66, 83=10.07, 84=32.68, 85=15.98, 86=34.83, 87=13.03, 88=18.28, 89=24.71, 90=21.16, 91=28.97, 92=22.

In [108]:
from vega_datasets import data

In [111]:
weather = data.seattle_weather()

In [112]:
weather = spark.createDataFrame(weather)

In [113]:
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [115]:
weather.withColumn('temp_min_F',
                   weather.temp_min * (9/5)+32
).show(2)

+-------------------+-------------+--------+--------+----+-------+----------+
|               date|precipitation|temp_max|temp_min|wind|weather|temp_min_F|
+-------------------+-------------+--------+--------+----+-------+----------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|      41.0|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|     37.04|
+-------------------+-------------+--------+--------+----+-------+----------+
only showing top 2 rows



In [117]:
weather.groupby('weather').sum('precipitation').show()

+-------+------------------+
|weather|sum(precipitation)|
+-------+------------------+
|drizzle|               1.0|
|   rain|1321.8000000000006|
|    sun|239.39999999999995|
|   snow|             208.1|
|    fog|2655.7000000000003|
+-------+------------------+



In [118]:
weather.groupby('weather').sum('wind').show()

+-------+------------------+
|weather|         sum(wind)|
+-------+------------------+
|drizzle|             130.7|
|   rain| 950.9999999999998|
|    sun|2135.4999999999995|
|   snow|             101.1|
|    fog|            1417.0|
+-------+------------------+



In [122]:
weather.withColumn(
    'year',
    F.year(weather.date)
).groupby(
'year'
).agg(
(F.mean('wind')).alias('mean_wind'),
    (F.mean('wind')).alias('total_wind')
).sort(F.col('mean_wind').desc()).first()

Row(year=2012, mean_wind=3.400819672131148, total_wind=3.400819672131148)

In [124]:
weather.filter(F.month(weather.date==1)).groupby('weather').count()

AnalysisException: [DATATYPE_MISMATCH.BINARY_OP_DIFF_TYPES] Cannot resolve "(date = 1)" due to data type mismatch: the left and right operands of the binary operator have incompatible types ("TIMESTAMP" and "INT").;
'Filter month((date#4981 = 1))
+- LogicalRDD [date#4981, precipitation#4982, temp_max#4983, temp_min#4984, wind#4985, weather#4986], false


23/05/19 12:48:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1013991 ms exceeds timeout 120000 ms
23/05/19 12:48:32 WARN SparkContext: Killing executors is not supported by current scheduler.
23/05/19 12:48:38 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.

23/05/19 12:48:48 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/05/19 12:48:58 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/05/19 12:49:08 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

...

23/05/19 13:38:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/05/19 13:38:59 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2088)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j

23/05/19 13:39:19 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2088)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j

23/05/19 13:39:29 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/05/19 13:39:39 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

In [126]:
weather.filter(
F.month(weather.date)==7).filter(F.year(weather.date)>2012).filter((F.year.date)<2015).agg(F.mean(temp_min.show()))

ConnectionRefusedError: [Errno 61] Connection refused

In [128]:
weather.filter((F.quarter(weather.date)==3).filter(F.year(weather.date)==2015)).select(F.when(weather.weather=='rain',1).otherwise(0)).mean().show()

ConnectionRefusedError: [Errno 61] Connection refused

In [129]:
weather.withColumn('year',F.year(weather.date)).withColumn('has_rained',weather.percipitation>0).show()

ConnectionRefusedError: [Errno 61] Connection refused